## Notebook for reading in data and anaylzing:

In [23]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [24]:
def read_data(file):
    data = pd.read_csv(file)
    return data


In [33]:
df = read_data('EEG Data 2025SP CMP SC 4540 01.csv')
print(df.head())
print("num of samples", df.shape[0])
print("num of features", df.shape[1]-1)

X = df.iloc[:, :14].values
y = df.iloc[:, 14].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred_log = log_reg.predict(X_test)

accuracy_log = accuracy_score(y_test, y_pred_log)
print(f"Logistic Regression Accuracy: {accuracy_log:.4f}")

X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).reshape(-1, 1).float()
X_test_tensor = torch.from_numpy(X_test).float()

   SubjectID  VideoID  Attention  Mediation    Raw      Delta     Theta  \
0        0.0      0.0       56.0       43.0  278.0   301963.0   90612.0   
1        0.0      0.0       40.0       35.0  -50.0    73787.0   28083.0   
2        0.0      0.0       47.0       48.0  101.0   758353.0  383745.0   
3        0.0      0.0       47.0       57.0   -5.0  2012240.0  129350.0   
4        0.0      0.0       44.0       53.0   -8.0  1005145.0  354328.0   

     Alpha1   Alpha2    Beta1     Beta2   Gamma1   Gamma2  predefinedlabel  \
0   33735.0  23991.0  27946.0   45097.0  33228.0   8293.0              0.0   
1    1439.0   2240.0   2746.0    3687.0   5293.0   2740.0              0.0   
2  201999.0  62107.0  36293.0  130536.0  57243.0  25354.0              0.0   
3   61236.0  17084.0  11488.0   62462.0  49960.0  33932.0              0.0   
4   37102.0  88881.0  45307.0   99603.0  44790.0  29749.0              0.0   

   user-definedlabeln  
0                 0.0  
1                 0.0  
2       

/opt/anaconda3/envs/ml_env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## The features which are inputs are the columns 0->14 and the last one is the target or label. This is a classification problem since the lables are labled 0 or 1 showing binary classification depending on the features. As a result this is used to find a line with the features and lables using a loss function to determine the line that best fit for predicting whether it is 0 or 1

In [27]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(14, 8)
        self.layer2 = nn.Linear(8, 1)  
        self.sigmoid = nn.Sigmoid()    
        
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.sigmoid(self.layer2(x))
        return x


In [28]:
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [29]:
epochs = 1000
for epoch in range(epochs):
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 20 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 20/1000, Loss: 39.2005
Epoch 40/1000, Loss: 34.1476
Epoch 60/1000, Loss: 32.1316
Epoch 80/1000, Loss: 26.4031
Epoch 100/1000, Loss: 18.0124
Epoch 120/1000, Loss: 8.4271
Epoch 140/1000, Loss: 1.7500
Epoch 160/1000, Loss: 0.7017
Epoch 180/1000, Loss: 0.6921
Epoch 200/1000, Loss: 0.6921
Epoch 220/1000, Loss: 0.6921
Epoch 240/1000, Loss: 0.6921
Epoch 260/1000, Loss: 0.6921
Epoch 280/1000, Loss: 0.6921
Epoch 300/1000, Loss: 0.6921
Epoch 320/1000, Loss: 0.6921
Epoch 340/1000, Loss: 0.6921
Epoch 360/1000, Loss: 0.6921
Epoch 380/1000, Loss: 0.6921
Epoch 400/1000, Loss: 0.6921
Epoch 420/1000, Loss: 0.6921
Epoch 440/1000, Loss: 0.6921
Epoch 460/1000, Loss: 0.6921
Epoch 480/1000, Loss: 0.6921
Epoch 500/1000, Loss: 0.6921
Epoch 520/1000, Loss: 0.6921
Epoch 540/1000, Loss: 0.6921
Epoch 560/1000, Loss: 0.6921
Epoch 580/1000, Loss: 0.6921
Epoch 600/1000, Loss: 0.6921
Epoch 620/1000, Loss: 0.6921
Epoch 640/1000, Loss: 0.6921
Epoch 660/1000, Loss: 0.6921
Epoch 680/1000, Loss: 0.6921
Epoch 700/100

In [30]:
model.eval()
with torch.no_grad():
    y_pred_nn = model(X_test_tensor)
    y_pred_nn = (y_pred_nn > 0.5).float().numpy().flatten()

In [31]:
accuracy_nn = accuracy_score(y_test, y_pred_nn)
print(f"Neural Network Accuracy: {accuracy_nn:.4f}")

Neural Network Accuracy: 0.4943


In [32]:
print("\nModel Comparison:")
print(f"Logistic Regression: {accuracy_log:.4f}")
print(f"Neural Network: {accuracy_nn:.4f}")


Model Comparison:
Logistic Regression: 0.6009
Neural Network: 0.4943
